<a href="https://colab.research.google.com/github/gyuminpk/Playground_solver/blob/main/%EC%88%AB%EC%9E%90%EB%A7%9E%EC%B6%94%EA%B8%B0_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import copy

# -----------------------------
# 문제 정의
# -----------------------------

problem_matrix = np.array([
    [0, 1, 0, 0, 3, 0, 3, 0],
    [0, 3, 2, 2, 1, 3, 2, 2],
    [1, 2, 0, 3, 0, 1, 0, 0],
    [3, 1, 0, 0, 1, 1, 0, 2],
    [3, 2, 1, 0, 1, 2, 2, 2],
    [1, 2, 3, 1, 2, 3, 2, 0],
    [0, 1, 2, 0, 3, 0, 3, 0],
    [3, 1, 2, 2, 2, 1, 2, 0]
])

target_answer_matrix = np.array([
    [5, 3, 4],
    [7, 4, 7],
    [8, 6, 7]
])

In [6]:


# -----------------------------
# 2x2 블록 회전
# -----------------------------
def rotate_block_clockwise(mat, r, c):
    temp = mat[r:r+2, c:c+2].copy()
    mat[r, c]     = temp[1,0]
    mat[r, c+1]   = temp[0,0]
    mat[r+1, c+1] = temp[0,1]
    mat[r+1, c]   = temp[1,1]

# -----------------------------
# answer cell 계산
# -----------------------------
def compute_answer_cell(mat, i, j):
    return (
        mat[2*i+1, 2*j+1] +
        mat[2*i+1, 2*j+2] +
        mat[2*i+2, 2*j+1] +
        mat[2*i+2, 2*j+2]
    )

# -----------------------------
# answer cell이 완성됐는지 체크
# -----------------------------
def is_answer_cell_complete(i, j, block_idx):
    required_blocks = [
        ((2*i)//2)*4 + ((2*j)//2),
        ((2*i)//2)*4 + ((2*j+2)//2),
        ((2*i+2)//2)*4 + ((2*j)//2),
        ((2*i+2)//2)*4 + ((2*j+2)//2),
    ]
    return max(required_blocks) <= block_idx

# -----------------------------
# DFS (완전 & 안전)
# -----------------------------
def solve(mat, block_idx, rotations):
    if block_idx == 16:
        return True

    r = (block_idx // 4) * 2
    c = (block_idx % 4) * 2

    for rot in range(4):
        # 회전 적용
        for _ in range(rot):
            rotate_block_clockwise(mat, r, c)
        rotations[block_idx] = rot

        # 완성된 answer cell만 검사
        valid = True
        for i in range(3):
            for j in range(3):
                if is_answer_cell_complete(i, j, block_idx):
                    if compute_answer_cell(mat, i, j) != target_answer_matrix[i, j]:
                        valid = False
                        break
            if not valid:
                break

        if valid:
            if solve(mat, block_idx + 1, rotations):
                return True

        # 원상복구 (역회전)
        for _ in range((4 - rot) % 4):
            rotate_block_clockwise(mat, r, c)

    return False

# -----------------------------
# 실행
# -----------------------------
rotations = [0] * 16
mat = problem_matrix.copy()

if solve(mat, 0, rotations):
    print("✅ Solution found!")
    print("Rotations:", np.reshape(np.array(rotations), (4, 4)))
else:
    print("❌ No solution")


✅ Solution found!
Rotations: [[0 2 1 3]
 [0 2 0 1]
 [0 0 2 0]
 [0 1 2 1]]
